# XArray THREDDS Problem Description
This notebook contains a description of a common error find while using the [_.load()_](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.load.html) function in [_xarray_](https://docs.xarray.dev/en/latest/index.html) package while connected in a THREDDS server to load data locally.

In this example, outputs from a global ocean model ([HYCOM](https://www.hycom.org/)) are acessed through a THREDDS server. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray

In [3]:
def convert_to_360(lon):
    return float(np.remainder(lon + 360, 360))

In [4]:
###File Location and Domain Definition
url = "https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0"
ds = xr.open_dataset(url, drop_variables="tau")

### Domain Specification
lonmin=-54
lonmax=-31
latmin=-36
latmax=7
selvar = 'water_u'

da = ds[selvar].sel(lon=slice(convert_to_360(lonmin), convert_to_360(lonmax)), lat=slice(latmin,latmax))

In [6]:
#ds[selvar].sel(x=slice(convert_to_360(lonmin), convert_to_360(lonmax)), y=slice(latmin,latmax))

In [6]:
unit = 1e9
print(f'The present dataset is about {np.round(da.size/unit)}GB in size')
print(f'The amount of data present in a single level is about {da.sel(depth=0).size/unit}GB')

The present dataset is about 207.0GB in size
The amount of data present in a single level is about 5.184116352GB


In [14]:
test1 = da.sel(depth=0).load()

oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 0:16728,0:0,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 0:16728,0:0,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 0:16728,0:0,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 0:16728,0:0,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "opendap.dap.DataReadException: Inconsistent array length read: 1165128303 != 1914731274; water_u -- 0:16728,0:0,1100:2175,3825:4112";
}"oc_open: server

RuntimeError: NetCDF: file not found

In [8]:
test2 = da.sel(time=slice("2024-01-01", "2024-01-10"))
print(f'The present dataset is about {np.round(test2.size/unit)}GB in size'

The present dataset is about 1.0GB in size


In [ ]:
test2 = da.sel(time=slice("2024-01-01", "2024-01-10")).load()

oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 14821:14900,0:39,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 14821:14900,0:39,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 14821:14900,0:39,1100:2175,3825:4112";
}"oc_open: server error retrieving url: code=? message="Error {
    code = 500;
    message = "java.net.SocketTimeoutException: Read timed out; water_u -- 14821:14900,0:39,1100:2175,3825:4112";
}"

The runtime errors are very common while trying to load a dataset as we can see by the above example and also by the print screen below, where the red poligons highlight the errors related with donwloading data in the [HYCOM forum](https://groups.google.com/a/hycom.org/g/forum?pli=1) 

![Minha Imagem](hycom_forum.png)

## OceanHackWeek 2024 Project
Therefore, in order to optimize the access of data trough a THREDDS server, during the [OceanHackWeek 2024](https://github.com/oceanhackweek), we develop a new _xarray accessor_ to facilitate the access to this data without runtime errors. For those who are not familiar with the _xarray accessor_, we reccomend their documentation in their own [website](https://docs.xarray.dev/en/stable/internals/extending-xarray.html)